In [35]:
import numpy as np
import sympy as sp

# 设置 J 和 L
J = 2  # 行数
L = 2  # 列数

# 定义符号变量 X_ij
X = sp.symbols('X0:%d' % (J * L))  # 创建一个含有 J*L 个符号变量的列表

# 将 X 转换为一个 JxL 矩阵
X_matrix = sp.Matrix(J, L, lambda i, j: X[i * L + j])

print(X_matrix)

# 构造 M(X) 的元素形式
def M_ij(X_matrix, j, l, J, L):
    # 计算 M(X) 的每个元素
    comp_time = 1
    comm_time = 10
    prod_term = 1
    for i in range(J):
        numerator = sum([comp_time + comm_time * X_matrix[i, k] for k in range(L)])  # ∑ (1 + X_{j,l})
        denominator = numerator - comm_time * X_matrix[i, l]  # ∑ (1 + X_{j,l}) - X_{i,l} * ||i ≠ j||
        if i != j: ## i 和 j 不在同一行
            prod_term *= (numerator / denominator)
    return prod_term

# 构造 M(X) 对 X 的雅可比矩阵
M_matrix = sp.Matrix(J, L, lambda i, j: M_ij(X_matrix, i, j, J, L))
print(M_matrix)
# jacobian_matrix = sp.Matrix(J * L, J * L, lambda i, j: 
#     (sp.diff(M_matrix[i, j], X_matrix[i, j]) for k in range(J) for l in range(L)))
jacobian_matrix = sp.zeros(J * L, J * L)
for i in range(J):
    for j in range(L):
        for k in range(J):
            for l in range(L):
                # M_matrix[i,j] 对 X_matrix[k,l] 的偏导数
                jacobian_matrix[i * L + j, k * L + l] = sp.diff(M_matrix[i, j], X_matrix[k, l])
# 将符号矩阵中的 X 替换为 numpy 数值（例如: X_matrix = np.ones((J, L)))
X_values = np.ones((J, L))

# 将 X_matrix 中的符号变量转换为数字矩阵
subs_dict = {X_matrix[i, j]: X_values[i, j] for i in range(J) for j in range(L)}

# 计算并输出在 X = np.ones((J, L)) 处的雅可比矩阵
jacobian_matrix_numeric = np.array([[jacobian_matrix[i, j].subs(subs_dict) for j in range(J * L)] for i in range(J * L)], dtype=np.float64)

print("在 X = np.ones(J, L) 处的雅可比矩阵：")
print(jacobian_matrix_numeric)
# 计算特征值
eigenvalues = np.linalg.eigvals(jacobian_matrix_numeric)
# 计算谱半径，取特征值的最大绝对值
print(eigenvalues)
spectral_radius = max(abs(eigenvalues))

print("谱半径：", spectral_radius)

Matrix([[X0, X1], [X2, X3]])
Matrix([[(10*X2 + 10*X3 + 2)/(10*X3 + 2), (10*X2 + 10*X3 + 2)/(10*X2 + 2)], [(10*X0 + 10*X1 + 2)/(10*X1 + 2), (10*X0 + 10*X1 + 2)/(10*X0 + 2)]])
在 X = np.ones(J, L) 处的雅可比矩阵：
[[ 0.          0.          0.83333333 -0.69444444]
 [ 0.          0.         -0.69444444  0.83333333]
 [ 0.83333333 -0.69444444  0.          0.        ]
 [-0.69444444  0.83333333  0.          0.        ]]
[ 1.52777778 -1.52777778  0.13888889 -0.13888889]
谱半径： 1.5277777777777788
